### Consumo de combustible: Análisis Exploratorio

In [9]:
import pandas as pd

consumo_2024 = pd.read_excel(
    './Datos_consumos/CONSUMO-HIDROCARBUROS-2024-12.xlsx',
    sheet_name='CONSUMO',
    header=6,
    parse_dates=['Fecha']
)

consumo_2025 = pd.read_excel(
    './Datos_consumos/VENTAS-HIDROCARBUROS-2025-05.xlsx',
    sheet_name='VENTAS_IMP',
    header=6,
    parse_dates=['Fecha']
)

consumo = pd.concat([consumo_2024, consumo_2025], ignore_index=True)
consumo.sort_values('Fecha', inplace=True)

consumo.set_index('Fecha', inplace=True)

diesel_cols = [c for c in consumo.columns if 'Diesel' in c]
consumo['Diesel'] = consumo[diesel_cols].sum(axis=1)

df_consumo = consumo.rename(columns={
    'Gasolina regular': 'Gasolina Regular',
    'Gasolina superior': 'Gasolina Super',
    'Gas licuado de petróleo': 'Gas Licuado'
})[['Gasolina Regular', 'Gasolina Super', 'Diesel', 'Gas Licuado']]

df_consumo.head()


,Gasolina Regular,Gasolina Super,Diesel,Gas Licuado
Fecha,,,,
2000-01-01 00:00:00,202645.20,308156.82,634667.06,194410.476190
2000-02-01 00:00:00,205530.96,307766.31,642380.66,174710.552381
2000-03-01 00:00:00,229499.56,331910.29,699807.25,189234.066667
2000-04-01 00:00:00,210680.40,315648.08,586803.98,174330.607143
2000-05-01 00:00:00,208164.34,319667.97,656948.20,191745.147619


In [ ]:
import pandas as pd

df_consumo.index = pd.to_datetime(df_consumo.index, errors='coerce')

df_consumo = df_consumo[df_consumo.index.notna()]

df_consumo = df_consumo[~df_consumo.index.duplicated(keep='first')]

df_consumo = df_consumo[df_consumo.index <= '2025-05-01']

print(df_consumo.shape)  
print("Desde", df_consumo.index.min(), "hasta", df_consumo.index.max())


(305, 4)
Desde 2000-01-01 00:00:00 hasta 2025-05-01 00:00:00


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for ax, col in zip(axes.flatten(), df_consumo.columns):
    sns.histplot(df_consumo[col], bins=20, kde=True, ax=ax)
    ax.set_title(f'Distribución de {col}')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
sns.boxplot(data=df_consumo, orient='h')
plt.title('Boxplot comparativo de consumos')
plt.show()

corr = df_consumo.corr()
print("Matriz de correlación:\n", corr, "\n")

plt.figure(figsize=(6, 5))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Heatmap de correlaciones')
plt.show()

pares = [('Gasolina Regular','Gasolina Super'),
         ('Gasolina Regular','Diesel'),
         ('Diesel','Gas Licuado')]

for x, y in pares:
    plt.figure(figsize=(5, 4))
    sns.regplot(x=df_consumo[x], y=df_consumo[y], line_kws={'color':'red'})
    plt.xlabel(x); plt.ylabel(y)
    plt.title(f'{y} vs {x}')
    plt.tight_layout()
    plt.show()
